FINE TUNING SEGMENTATION MODEL

In [ ]:
!pip uninstall -y torchvision torchmetrics pytorch-lightning lightning-fabric

Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126


In [ ]:
# 1. Surgical version fix for the HuggingFace conflict
!pip install -q "huggingface-hub<1.0" "transformers>=4.40.0"

# 2. Re-install the core audio tools
!pip install -q pyannote.audio==3.1.1 soundfile torch==2.6.0 torchaudio==2.6.0

# 3. Standard Kernel Reset (Mandatory)
import os
os._exit(0)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.6/766.6 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 142.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 913.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00


In [ ]:
!unzip rttm_maithli.zip

Archive:  rttm_maithli.zip
   creating: rttm/
  inflating: rttm/A1.rttm            
  inflating: rttm/A10.rttm           
  inflating: rttm/A100.RTTM          
  inflating: rttm/A11.rttm           
  inflating: rttm/A12.rttm           
  inflating: rttm/A13.rttm           
  inflating: rttm/A14.rttm           
  inflating: rttm/A15.rttm           
  inflating: rttm/A16.rttm           
  inflating: rttm/A17.rttm           
  inflating: rttm/A18.rttm           
  inflating: rttm/A19.rttm           
  inflating: rttm/A2.rttm            
  inflating: rttm/A20.rttm           
  inflating: rttm/A21.rttm           
  inflating: rttm/A22.rttm           
  inflating: rttm/A23.rttm           
  inflating: rttm/A24.rttm           
  inflating: rttm/A25.rttm           
  inflating: rttm/A26.rttm           
  inflating: rttm/A27.rttm           
  inflating: rttm/A28.rttm           
  inflating: rttm/A29.rttm           
  inflating: rttm/A3.rttm            
  inflating: rttm/A30.rttm           
  in

In [ ]:
!unzip audio_maithli.zip

Archive:  audio_maithli.zip
   creating: audios/
  inflating: audios/A1.wav           
  inflating: audios/A10.wav          
  inflating: audios/A100.wav         
  inflating: audios/A11.wav          
  inflating: audios/A12.wav          
  inflating: audios/A13.wav          
  inflating: audios/A14.wav          
  inflating: audios/A15.wav          
  inflating: audios/A16.wav          
  inflating: audios/A17.wav          
  inflating: audios/A18.wav          
  inflating: audios/A19.wav          
  inflating: audios/A2.wav           
  inflating: audios/A20.wav          
  inflating: audios/A21.wav          
  inflating: audios/A22.wav          
  inflating: audios/A23.wav          
  inflating: audios/A24.wav          
  inflating: audios/A25.wav          
  inflating: audios/A26.wav          
  inflating: audios/A27.wav          
  inflating: audios/A28.wav          
  inflating: audios/A29.wav          
  inflating: audios/A3.wav           
  inflating: audios/A30.wav          
 

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
# Run this and paste your 'WRITE' token into the box that appears
notebook_login()

In [ ]:
if not hasattr(np, "NaN"): np.NaN = np.nan

In [ ]:
import torch, os, re, numpy as np, soundfile as sf
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from pyannote.audio import Model

# --- 1. THE 2026 STABILITY LAYER ---

# Fix for Torch 2.6+ security settings
def safe_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return torch.serialization.load(*args, **kwargs)
torch.load = safe_load

class IndustrialHindiDataset(Dataset):
    def __init__(self, audio_dir, rttm_dir, duration=2.0, sr=16000):
        self.audio_dir, self.rttm_dir = audio_dir, rttm_dir
        self.duration, self.sr = duration, sr

        def nat_sort(s):
            return [int(t) if t.isdigit() else t.lower()
                    for t in re.split('([0-9]+)', s)]

        audio_files = sorted(
            [f for f in os.listdir(audio_dir) if f.endswith(".wav")],
            key=nat_sort
        )
        rttm_files = {
            os.path.splitext(f)[0]: f
            for f in os.listdir(rttm_dir)
            if f.lower().endswith(".rttm")
        }

        # 🔒 SAFE PAIRING
        self.pairs = []
        for wav in audio_files:
            base = os.path.splitext(wav)[0]
            if base in rttm_files:
                self.pairs.append((wav, rttm_files[base]))

        assert len(self.pairs) > 0, "❌ No matching audio–RTTM pairs found"

        print(f"✅ Loaded {len(self.pairs)} aligned audio–RTTM pairs")

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        audio_file, rttm_file = self.pairs[idx]

        wav_path = os.path.join(self.audio_dir, audio_file)
        data, _ = sf.read(wav_path, frames=int(self.sr * self.duration))

        # Loudness normalization
        if np.max(np.abs(data)) > 0:
            data = data / np.max(np.abs(data))

        if data.ndim > 1:
            data = data[:, 0]

        if len(data) < int(self.sr * self.duration):
            data = np.pad(
                data,
                (0, int(self.sr * self.duration) - len(data))
            )

        waveform = torch.from_numpy(data).float().unsqueeze(0)
        target = torch.zeros((115, 7))

        rttm_path = os.path.join(self.rttm_dir, rttm_file)
        with open(rttm_path, "r") as f:
            for line in f:
                p = line.strip().split()
                if len(p) >= 8:
                    start, dur = float(p[3]), float(p[4])
                    if start < self.duration:
                        start_f = int((start / self.duration) * 115)
                        end_f = int(
                            (min(start + dur, self.duration) / self.duration) * 115
                        )
                        spk_idx = int(re.search(r"\d+", p[7]).group()) - 1
                        if 0 <= spk_idx < 7:
                            target[start_f:end_f, spk_idx] = 1.0

        return waveform, target


# --- 3. THE TRAINING ENGINE ---
def run_final_training(hf_token, epochs=20):
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🎬 Environment Cleaned. Training on {DEVICE}...")

    model = Model.from_pretrained("pyannote/segmentation-3.0", use_auth_token=hf_token).to(DEVICE)
    dataset = IndustrialHindiDataset("/content/audios", "/content/rttm")
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    criterion = nn.BCEWithLogitsLoss()

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for x, y in dataloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"✅ Epoch {epoch} | Avg Loss: {epoch_loss/len(dataloader):.4f}")

    torch.save(model.state_dict(), "hindi_finetuned_final.pt")
    print("🎉 SUCCESS: Weights saved as 'hindi_finetuned_final.pt'")

run_final_training(hf_token)

🎬 Environment Cleaned. Training on cuda...
✅ Loaded 100 aligned audio–RTTM pairs
✅ Epoch 0 | Avg Loss: 0.2303
✅ Epoch 1 | Avg Loss: 0.2241
✅ Epoch 2 | Avg Loss: 0.2299
✅ Epoch 3 | Avg Loss: 0.2280
✅ Epoch 4 | Avg Loss: 0.2307
✅ Epoch 5 | Avg Loss: 0.2332
✅ Epoch 6 | Avg Loss: 0.2206
✅ Epoch 7 | Avg Loss: 0.2245
✅ Epoch 8 | Avg Loss: 0.2188
✅ Epoch 9 | Avg Loss: 0.2212
✅ Epoch 10 | Avg Loss: 0.2299
✅ Epoch 11 | Avg Loss: 0.2175
✅ Epoch 12 | Avg Loss: 0.2218
✅ Epoch 13 | Avg Loss: 0.2150
✅ Epoch 14 | Avg Loss: 0.2213
✅ Epoch 15 | Avg Loss: 0.2115
✅ Epoch 16 | Avg Loss: 0.2113
✅ Epoch 17 | Avg Loss: 0.2114
✅ Epoch 18 | Avg Loss: 0.2138
✅ Epoch 19 | Avg Loss: 0.2111
🎉 SUCCESS: Weights saved as 'hindi_finetuned_final.pt'


In [ ]:
USING THE FINE TUNE SEGMENTATION MODEL IN PIPELINE

In [ ]:
!pip install torch==2.2.2+cu118 torchaudio==2.2.2+cu118 \
  -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:
!pip uninstall -y torch torchvision torchaudio


In [ ]:
!pip install torch==2.2.2+cu118 torchvision==0.17.2+cu118 torchaudio==2.2.2+cu118 \
  -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:
!pip install pyannote.audio==3.1.1 \
  speechbrain==0.5.16 \
  numpy==1.26.4 \
  librosa==0.10.1 \
  soundfile==0.12.1 \
  scipy==1.11.4 \
  scikit-learn==1.3.2 \
  pytorch-lightning==2.1.3 \
  einops==0.7.0 \
  omegaconf==2.3.0


In [ ]:
import torch, torchaudio, torchvision, numpy, pyannote.audio

print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("torchaudio:", torchaudio.__version__)
print("numpy:", numpy.__version__)
print("pyannote:", pyannote.audio.__version__)


In [ ]:
from huggingface_hub import login
login()


In [ ]:
!ls



In [ ]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=True
)

from pyannote.audio import Model

# load base segmentation architecture
segmentation_model = Model.from_pretrained(
    "pyannote/segmentation-3.0",
    use_auth_token=True
)

# load your fine-tuned weights
state_dict = torch.load("/content/hindi_finetuned_20.pt", map_location="cpu")
segmentation_model.load_state_dict(state_dict)
segmentation_model.eval()

# 🔥 THIS IS THE KEY LINE
pipeline.segmentation.model = segmentation_model




In [ ]:
diarization = pipeline("/content/t2.wav")

for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"{speaker}: {turn.start:.2f}s --> {turn.end:.2f}s")


In [ ]:
diarization = pipeline("/content/A92.wav")

for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"{speaker}: {turn.start:.2f}s --> {turn.end:.2f}s")


In [ ]:
diarization = pipeline("/content/A96.wav")

for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"{speaker}: {turn.start:.2f}s --> {turn.end:.2f}s")


In [ ]:
diarization = pipeline("/content/A96.wav")

for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"{speaker}: {turn.start:.2f}s --> {turn.end:.2f}s")


In [ ]:
from pyannote.metrics.diarization import DiarizationErrorRate
from pyannote.core import Segment, Annotation

# Example: create reference Annotation
reference = Annotation()
reference[Segment(0.0, 5.25)] = "Speaker_0"
reference[Segment(5.25, 12.80)] = "Speaker_1"

# Example: predicted Annotation (from your pipeline)
prediction = Annotation()
prediction[Segment(0.0, 5.20)] = "Speaker_0"
prediction[Segment(5.20, 12.85)] = "Speaker_1"

metric = DiarizationErrorRate()
der_value = metric(reference, prediction)
print(f"DER: {der_value:.3f}")  # e.g., 0.12 → 12% error


UI CODE

In [ ]:
import streamlit as st
import tempfile
import os

import torchaudio
from dotenv import load_dotenv
from pyannote.audio import Pipeline

# ===============================
# Environment
# ===============================
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")

st.set_page_config(page_title="Speaker Diarization", layout="centered")
st.title("🎙️ Speaker Diarization")
st.caption("Upload a WAV file to see speaker-wise segments")

# ===============================
# WAV-only upload
# ===============================
uploaded_file = st.file_uploader(
    "Upload WAV audio file",
    type=["wav"]
)

# ===============================
# Audio conversion (safety)
# ===============================
def convert_audio_to_16k_mono(input_path, output_path):
    waveform, sample_rate = torchaudio.load(input_path)

    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(
            orig_freq=sample_rate,
            new_freq=16000
        )
        waveform = resampler(waveform)

    torchaudio.save(output_path, waveform, 16000)

# ===============================
# Load diarization pipeline
# ===============================
@st.cache_resource
def load_pipeline():
    return Pipeline.from_pretrained(
        "pyannote/speaker-diarization",
        use_auth_token=HF_TOKEN
    )

pipeline = load_pipeline()

# ===============================
# Run diarization
# ===============================
if uploaded_file is not None:
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as raw:
        raw.write(uploaded_file.read())
        raw_path = raw.name

    processed_path = raw_path.replace(".wav", "_16k.wav")
    convert_audio_to_16k_mono(raw_path, processed_path)

    # Audio player
    st.audio(processed_path)

    with st.spinner("Running speaker diarization..."):
        diarization = pipeline(
            processed_path,
            min_speakers=1,
            max_speakers=3
        )

    st.success("Diarization complete")

    # ===============================
    # Caption-style speaker timeline
    # ===============================
    st.subheader("📝 Speaker Timeline")

    speaker_map = {}
    speaker_count = 1

    for turn, _, speaker in diarization.itertracks(yield_label=True):
        if speaker not in speaker_map:
            speaker_map[speaker] = f"Speaker {speaker_count}"
            speaker_count += 1

        label = speaker_map[speaker]
        start = f"{turn.start:.2f}"
        end = f"{turn.end:.2f}"

        st.markdown(
            f"""
            **[{start}s – {end}s]**
            🔊 {label}
            ---
            """
        )